***building with RunnablePassthrough***

In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="gemma:2b")

model

ChatOllama(model='gemma:2b')

In [2]:
from langchain_core.prompts import PromptTemplate

prompt1 = PromptTemplate(
    template="""
        create a joke on the topic of: {topic}
    """,
    input_variables=['topic']
)

prompt1

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        create a joke on the topic of: {topic}\n    ')

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

parser

StrOutputParser()

In [7]:
from langchain_core.runnables import RunnableSequence

sequential_chain = RunnableSequence(prompt1, model, parser)

sequential_chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='\n        create a joke on the topic of: {topic}\n    ')
| ChatOllama(model='gemma:2b')
| StrOutputParser()

In [10]:
prompt2 = PromptTemplate(
    template="""
        explain the joke: {text}, with proper things like what we tried to laugh, the sarcasm present in joke, etc
        in simple and short way!
    """,
    input_variables=['text']
)

prompt2

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n        explain the joke: {text}, with proper things like what we tried to laugh, the sarcasm present in joke, etc\n        in simple and short way!\n    ')

In [6]:
user_input = input("enter the topic: \n")


enter the topic: 
 Taj Mahal


In [11]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explain': RunnableSequence(prompt2, model, parser)
})

parallel_chain

{
  joke: RunnablePassthrough(),
  explain: PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n        explain the joke: {text}, with proper things like what we tried to laugh, the sarcasm present in joke, etc\n        in simple and short way!\n    ')
           | ChatOllama(model='gemma:2b')
           | StrOutputParser()
}

In [12]:
chain = sequential_chain | parallel_chain

response = chain.invoke({"topic":user_input})

response

{'joke': "What do you call a Taj Mahal that's too proud?\n\nA majestic monument of ego!",
 'explain': 'The joke is about a majestic monument, the Taj Mahal, that is too proud of itself. The word "ego" is used in a derogatory sense to describe something that is arrogant and self-centered. The joke suggests that the Taj Mahal is so proud of its beauty and grandeur that it is arrogant and disregards the feelings of those who admire it.'}

In [13]:
chain.get_graph().print_ascii()

                  +-------------+                  
                  | PromptInput |                  
                  +-------------+                  
                          *                        
                          *                        
                          *                        
                +----------------+                 
                | PromptTemplate |                 
                +----------------+                 
                          *                        
                          *                        
                          *                        
                  +------------+                   
                  | ChatOllama |                   
                  +------------+                   
                          *                        
                          *                        
                          *                        
                +-----------------+                
            